In [1]:
!pip install vaderSentiment textblob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import numpy as np



   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   --------------------------------- ------ 524.3/624.3 kB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 624.3/624.3 kB 2.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
class LexiconSentimentAnalyzer:
    
    def __init__(self):
        self.vader = SentimentIntensityAnalyzer()
        
        self.custom_lexicon = {
            'profit': 1, 'growth': 1, 'gain': 1, 'surge': 1,
            'loss': -1, 'decline': -1, 'fall': -1, 'drop': -1,
            'strong': 1, 'weak': -1, 'beat': 1, 'miss': -1,
            'positive': 1, 'negative': -1, 'risk': -1,
            'increase': 1, 'decrease': -1, 'bullish': 1,
            'bearish': -1, 'record': 1
        }
    
    def vader_sentiment(self, text):
        score = self.vader.polarity_scores(text)['compound']
        label = 'positive' if score > 0 else 'negative' if score < 0 else 'neutral'
        return score, label
    
    def textblob_sentiment(self, text):
        polarity = TextBlob(text).sentiment.polarity
        label = 'positive' if polarity > 0 else 'negative' if polarity < 0 else 'neutral'
        return polarity, label
    
    def custom_lexicon_sentiment(self, text):
        score = 0
        for word in text.lower().split():
            score += self.custom_lexicon.get(word, 0)
        return np.tanh(score)  # normalize to [-1, 1]
    
    def analyze(self, text):
        vader_score, vader_label = self.vader_sentiment(text)
        tb_score, tb_label = self.textblob_sentiment(text)
        custom_score = self.custom_lexicon_sentiment(text)
        
        ensemble = np.mean([vader_score, tb_score, custom_score])
        
        return {
            'text': text,
            'vader': {'compound': vader_score, 'label': vader_label},
            'textblob': {'polarity': tb_score, 'label': tb_label},
            'custom_score': custom_score,
            'ensemble_score': ensemble,
            'confidence': abs(ensemble)
        }


In [3]:
analyzer = LexiconSentimentAnalyzer()

analyzer.analyze("Excellent earnings beat expectations")


{'text': 'Excellent earnings beat expectations',
 'vader': {'compound': 0.5719, 'label': 'positive'},
 'textblob': {'polarity': 1.0, 'label': 'positive'},
 'custom_score': np.float64(0.7615941559557649),
 'ensemble_score': np.float64(0.7778313853185882),
 'confidence': np.float64(0.7778313853185882)}